# HULL TACTICAL MARKET PREDICTION - SUBMISSION NOTEBOOK

This notebook trains a Linear Regression model for tactical market prediction and sets up an inference server following the official competition format.

**Workflow:**
1. Load and prepare training data
2. Train Linear Regression model
3. Save model artifacts
4. Define prediction function
5. Start inference server

In [5]:
import os
import pandas as pd
import numpy as np
import joblib
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
try:
    import polars as pl
except ImportError:
    pl = None
    print("⚠ polars not installed (not needed for local testing)")

try:
    import kaggle_evaluation.default_inference_server as ke
except ImportError:
    ke = None
    print("⚠ kaggle_evaluation not installed (not needed for local testing)")

warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")

⚠ polars not installed (not needed for local testing)
⚠ kaggle_evaluation not installed (not needed for local testing)
✓ All libraries imported successfully


IF RUNNING ON KAGGLE, DONT FORGET TO CHANGE CONFIG

In [ ]:
# Configuration Kaggle Notebook Version
# CONFIG = {
#     'train_data_path': '/kaggle/input/hull-tactical-market-prediction/train.csv',
#     'input_dir': '/kaggle/input/hull-tactical-market-prediction/',
#     'model_save_path': '/tmp/model.joblib',
#     'scaler_save_path': '/tmp/scaler.joblib',
#     'features_save_path': '/tmp/feature_cols.joblib',
#     'target_column': 'forward_returns',
#     'prediction_bounds': (0.0, 2.0),
# }

import os
from pathlib import Path  # Add this import!

# Get the directory where this notebook is located
NOTEBOOK_DIR = Path.cwd()

PROJECT_DIR = NOTEBOOK_DIR.parent


# Configuration - For your setup
CONFIG = {
    'train_data_path': PROJECT_DIR / 'data' / 'train.csv',
    'test_data_path': PROJECT_DIR / 'data' / 'test.csv',
    'input_dir': PROJECT_DIR / 'data',
    
    # Save models in artifacts folder
    'model_save_path': PROJECT_DIR / 'artifacts' / 'model.joblib',
    'scaler_save_path': PROJECT_DIR / 'artifacts' / 'scaler.joblib',
    'features_save_path': PROJECT_DIR / 'artifacts' / 'feature_cols.joblib',
    
    'target_column': 'forward_returns',
    'prediction_bounds': (0.0, 2.0),
}

# Feature list (20 features)
FEATURE_COLS = [
    'V7', 'V13', 'E3', 'M4', 'P11', 'S2', 'I7', 'P2', 'P6', 'E17',
    'M12', 'M11', 'E2', 'I2', 'E9', 'P8', 'P5', 'I5', 'I9', 'V12'
]

print(f"✓ Configuration loaded")
print(f"✓ Features defined: {len(FEATURE_COLS)} features")

✓ Configuration loaded
✓ Features defined: 20 features


In [12]:
print("\n" + "="*70)
print("TRAINING PHASE")
print("="*70)

# Load training data
train_df = pd.read_csv(CONFIG['train_data_path'])
print(f"✓ Loaded training data: {train_df.shape}")
print(f"  - Rows: {train_df.shape[0]:,}")
print(f"  - Columns: {train_df.shape[1]}")


TRAINING PHASE
✓ Loaded training data: (9021, 98)
  - Rows: 9,021
  - Columns: 98


In [21]:
# Extract features and target
X_train = train_df[FEATURE_COLS].copy()
y_train = train_df[CONFIG['target_column']].copy()

# Handle missing values
X_train = X_train.ffill().bfill()

# Remove NaN rows
valid_idx = ~(X_train.isnull().any(axis=1) | y_train.isnull())
X_train = X_train[valid_idx]
y_train = y_train[valid_idx]

print(f"\nTraining samples after cleaning: {len(X_train)}")

#Scaling features

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)



Training samples after cleaning: 9021


In [23]:
# Train Linear Regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

train_r2 = model.score(X_train_scaled, y_train)

print(f"\n✓ Training R²: {train_r2:.6f}")
print(f"✓ Model trained with {len(FEATURE_COLS)} features")

# Save Model Artifacts
print(f"\n✓ Saving model artifacts...")

joblib.dump(model, str(CONFIG['model_save_path']))
joblib.dump(scaler, str(CONFIG['scaler_save_path']))
joblib.dump(FEATURE_COLS, str(CONFIG['features_save_path']))

print(f"  - Model saved to: {CONFIG['model_save_path']}")
print(f"  - Scaler saved to: {CONFIG['scaler_save_path']}")
print(f"  - Features saved to: {CONFIG['features_save_path']}")
print("✓ Model saved")


✓ Training R²: 0.013567
✓ Model trained with 20 features

✓ Saving model artifacts...


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Zehra Marziya Cengiz\\Desktop\\HullTactical\\notebooks\\artifacts\\model.joblib'